In [1]:
from vierlinden.data.loader import VierlindenDataProcessor
from vierlinden.config import model_output_path

target = ''

seed = 42

context_length = 50
prediction_length = 10
batch_size = 32
num_workers = 18

dp = VierlindenDataProcessor()
vierlinden_data = dp.load_processed_data()
vierlinden_data = dp.prepare_for_target(vierlinden_data, target)
vierlinden_data

e:\Users\Vipin\Documents\BHT\CalgoLab\RIWWER\.venv\lib\site-packages\pytorch_forecasting\models\base_model.py:30: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm
INFO:vierlinden.data.loader:Data loaded successfully from E:/Users/Vipin/Documents/BHT/CalgoLab/RIWWER/data
INFO:vierlinden.data.loader:Datetime processed successfully.
INFO:vierlinden.data.loader:Data loaded and processed successfully.


,Datetime,Herzog_Schieber_Position_pval,Herzog_Oberwasser_pval,Herzog_Unterwasser_pval,Herzog_Durchflußmenge_pval,Herzog_Berechnete_Durchflussmenge_pval,FLP_Hohenstand_Pumpensumpf_pval,FLP_P3_pval,FLP_P4_pval,FLP_P5_pval,...,Verlindenhof_Pumpe_1_pval,Verlindenhof_Pumpe_2_pval,Verlindenhof_Pumpe_3_pval,Niederschlag_mm,PV_15_Entleerung_RUEB_ival,PV_16_Regenueberlauf_Menge_ival,PV_18_Fuellstand_RUEB_1_ival,PV_19_Fuellstand_RUEB_2_ival,PV_20_Fuellstand_RUEB_3_ival,PV_25_Fuellstand_RRB_ival
0,2021-01-01 00:00:00,100.0,8.0761,5.8152,7.6009,7.5300,NaN,NaN,NaN,NaN,...,0.0000,0.0,1.0,0.00,0.098,0.0,3.16,3.08,2.72,1.47
1,2021-01-01 01:00:00,100.0,8.0841,5.3923,7.1753,8.0380,NaN,NaN,NaN,NaN,...,0.0000,0.0,1.0,0.00,0.099,0.0,3.16,3.08,2.72,1.47
2,2021-01-01 02:00:00,100.0,7.9639,5.1718,6.4808,7.7368,NaN,NaN,NaN,NaN,...,0.0000,0.0,1.0,0.00,0.096,0.0,3.16,3.08,2.72,1.47
3,2021-01-01 03:00:00,100.0,7.6138,5.0000,5.2214,6.1928,NaN,NaN,NaN,NaN,...,0.0000,0.0,1.0,0.00,0.098,0.0,3.16,3.08,2.72,1.47
4,2021-01-01 04:00:00,100.0,6.9667,4.5230,3.5886,4.3735,NaN,NaN,NaN,NaN,...,3.5338,0.0,1.0,0.00,0.098,0.0,3.16,3.08,2.72,1.47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26276,2023-12-31 20:00:00,100.0,9.6343,6.0298,0.0000,12.6065,91.5546,0.0,0.0,-1.0,...,0.0000,0.0,1.0,0.00,0.118,0.0,1.34,1.78,0.92,1.34
26277,2023-12-31 21:00:00,100.0,9.0974,5.9716,0.0000,10.9807,89.5880,0.0,0.0,-1.0,...,0.0000,0.0,1.0,0.00,0.120,0.0,1.35,1.78,0.92,1.34
26278,2023-12-31 22:00:00,100.0,8.7244,5.6111,0.0000,10.1204,90.6291,0.0,0.0,-1.0,...,0.0000,0.0,1.0,0.26,0.110,0.0,1.35,1.78,0.92,1.34
26279,2023-12-31 23:00:00,100.0,8.5567,5.2766,0.0000,10.0989,89.4323,0.0,0.0,-1.0,...,0.0000,0.0,1.0,0.07,0.112,0.0,1.36,1.78,0.92,1.34
